In [1]:
### Import libraries ----
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score


In [2]:
### Generate data ----
#!Python 1_Get_UCDP.py

In [3]:
### Load data ----
df_ucdp = pd.read_csv('ucdp_month.csv')
df_ucdp = df_ucdp.set_index("date")
df_ucdp

,Afghanistan,Albania,Algeria,Angola,Argentina,Armenia,Australia,Austria,Azerbaijan,Bahrain,...,Uganda,Ukraine,United Arab Emirates,United Kingdom,United States of America,Uzbekistan,Venezuela,Yemen (North Yemen),Zambia,Zimbabwe (Rhodesia)
date,,,,,,,,,,,,,,,,,,,,,
1989-01,1298.788462,0,0.0,300.273810,0,0.0,0,0,0.000000,0,...,0.000000,0.000000,0,3.0,0,0.0,0.000000,0.000000,0.0,0.0
1989-02,198.715385,0,0.0,427.809524,0,0.0,0,0,0.000000,0,...,0.000000,0.000000,0,5.0,0,0.0,21.897727,0.000000,0.0,0.0
1989-03,2211.050962,0,0.0,415.261905,0,0.0,0,0,0.000000,0,...,0.000000,0.000000,0,5.0,0,0.0,22.505682,0.000000,0.0,10.5
1989-04,510.730769,0,0.0,177.857143,0,0.0,0,0,0.000000,0,...,97.333333,0.000000,0,0.0,0,0.0,0.000000,0.000000,0.0,0.0
1989-05,821.833544,0,0.0,278.592437,0,0.0,0,0,0.000000,0,...,593.126667,0.000000,0,1.0,0,0.0,0.000000,0.000000,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09,813.026316,0,0.0,0.000000,0,1.0,0,0,0.857143,0,...,0.000000,15.082873,0,0.0,0,0.0,1.000000,3212.639990,0.0,0.0
2021-10,243.317982,0,0.0,0.000000,0,0.0,0,0,1.000000,0,...,0.000000,15.310635,0,0.0,0,0.0,0.000000,5431.433057,0.0,0.0
2021-11,80.500000,0,0.0,1.000000,0,1.0,0,0,14.000000,0,...,0.000000,13.151838,0,0.0,0,0.0,0.000000,5790.900966,0.0,0.0


In [4]:
### Prepare data -----
number_s=11          # 10 months sequences

# Normalization
scaler = MinMaxScaler(feature_range=(0,1))
df = pd.DataFrame(scaler.fit_transform(df_ucdp))

In [5]:
# Creation of the sequences
ts_seq=[]
for col in range(len(df.columns)):
    for i in range(number_s-1,len(df)):
        ts_seq.append(df.iloc[i-number_s+1:i+1,col])

In [6]:
# Creation of trainset, validation and testet
ts_seq=np.array(ts_seq)
ts_seq_l= ts_seq.reshape(len(df.columns),len(df.index)-number_s+1,number_s)

ts_seq_learn=ts_seq_l[:,:int(0.6*len(df)),:]
ts_seq_learn=ts_seq_learn.reshape(ts_seq_learn.shape[0]*ts_seq_learn.shape[1],number_s)

ts_seq_val=ts_seq_l[:,int(0.6*len(df)):int(0.8*len(df)),:]
ts_seq_val=ts_seq_val.reshape(ts_seq_val.shape[0]*ts_seq_val.shape[1],number_s)

ts_seq_test=ts_seq_l[:,int(0.8*len(df)):,:]
ts_seq_test=ts_seq_test.reshape(ts_seq_test.shape[0]*ts_seq_test.shape[1],number_s)

In [7]:
# Creation of input/output for each set
train_x = ts_seq_learn[:,:-1]
train_y = ts_seq_learn[:,-1]
val_x = ts_seq_val[:,:-1]
val_y = ts_seq_val[:,-1]
test_x = ts_seq_test[:,:-1]
test_y = ts_seq_test[:,-1]

In [ ]:
# Hyperparameter tuning -----

tuning = pd.DataFrame(columns=["n_trees", "min_split", "mse", "wmse"])

for n in [50, 100, 200, 500]:
    print(n)
    for min_split in [20, 30, 40]:
        print(min_split)
        model = RandomForestRegressor(n_estimators=n, min_samples_split=min_split).fit(train_x, train_y)
        pred = model.predict(val_x)
        mse = mean_squared_error(val_y, pred)
        wmse =  mean_squared_error(val_y,pred,sample_weight=val_y+1)
    
        # Save results
        para = pd.DataFrame({"n_trees": n,
                             "min_split": min_split,
                             "mse": mse, 
                             "wmse": wmse},
                             index=[n])
        tuning = tuning.append(para)
        tuning.reset_index(drop=True, inplace=True)

tuning.to_latex(f'tuning_random_forest.tex', index=False)
tuning

50
20
30
40
100
20
30
40
200
20
30
40
500
20
30
40


In [ ]:
### Sort
tuning.sort_values(by="wmse")

In [ ]:
### Final model ----
model = RandomForestRegressor(n_estimators=500, min_samples_split=20).fit(train_x, train_y)
pred = model.predict(test_x)
mse = mean_squared_error(test_y,pred)
print(mse)
weighted_mse =  mean_squared_error(test_y,pred,sample_weight=test_y+1)
print(weighted_mse)

In [ ]:
### Feature importance 
model.feature_importances_

In [ ]:
### Convert back to original dataframe ----
df_rf = pred.reshape((len(df.iloc[0,:]),int(len(ts_seq_test[:,:-1])/len(df.iloc[0,:]))))
df_rf = df_rf.T
df_nn = pd.DataFrame(df_rf)
df_rf = pd.DataFrame(scaler.inverse_transform(df_rf))
df_rf.columns = df_rf.columns
df_rf.index=df_ucdp.index[-70:]
df_rf.to_csv('preds_rf.csv', index=False)
df_rf.head(10)